# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [128]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [129]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.drop([467,484], inplace=True)
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [130]:
symbol= 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2721687440402,
 'week52high': 187.01,
 'week52low': 117.55,
 'week52highSplitAdjustOnly': 187.73,
 'week52lowSplitAdjustOnly': 117.17,
 'week52change': 0.38589194156755824,
 'sharesOutstanding': 16713739151,
 'float': 0,
 'avg10Volume': 94672011,
 'avg30Volume': 94672005,
 'day200MovingAvg': 164.23,
 'day50MovingAvg': 171.5,
 'employees': 154149,
 'ttmEPS': 6.31,
 'ttmDividendRate': 0.883217955733834,
 'dividendYield': 0.0053817629374903,
 'nextDividendDate': '',
 'exDividendDate': '2022-01-29',
 'nextEarningsDate': '2022-04-20',
 'peRatio': 27.965163960310868,
 'beta': 1.3444159848815298,
 'maxChangePercent': 63.56304772279407,
 'year5ChangePercent': 4.296911230749543,
 'year2ChangePercent': 1.3563158862571587,
 'year1ChangePercent': 0.3871537198850934,
 'ytdChangePercent': -0.0712908883919429,
 'month6ChangePercent': 0.11495362764412746,
 'month3ChangePercent': 0.01972146940869519,
 'month1ChangePercent': 0.03356948705910869,
 'day30ChangePe

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [131]:
data['year1ChangePercent']

0.3871537198850934

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [132]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [133]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats,quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    #print(data['AAPL']['stats'])
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ], index = my_columns
            ), ignore_index = True
        )
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,134.67,0.104573,N/A
1,AAL,18.15,-0.167863,N/A
2,AAP,213.49,0.308247,N/A
3,AAPL,165.10,0.378288,N/A
4,ABBV,150.54,0.475473,N/A
...,...,...,...,...
498,YUM,128.57,0.205013,N/A
499,ZBH,128.90,-0.236762,N/A
500,ZBRA,427.46,-0.165622,N/A
501,ZION,71.12,0.342532,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [134]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,83.45,2.28816,N/A
1,148,DVN,55.50,1.77289,N/A
2,344,NUE,131.44,1.1731,N/A
3,299,MCHP,73.75,1.0242,N/A
4,175,FANG,134.17,0.997183,N/A
5,317,MRO,22.54,0.967715,N/A
6,195,FTNT,336.48,0.946771,N/A
7,345,NVDA,250.07,0.836273,N/A
8,109,COP,93.98,0.777753,N/A
9,89,CF,81.90,0.767192,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [135]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio')
    i=0
    while(i==0):
        try:
            float(portfolio_size)
            i=1
        except:
            print("That's not a number.\nPlease try again")
            portfolio_size = input('Enter the size of your portfolio')
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio10000000
10000000


In [136]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,83.45,2.28816,2396
1,148,DVN,55.50,1.77289,3603
2,344,NUE,131.44,1.1731,1521
3,299,MCHP,73.75,1.0242,2711
4,175,FANG,134.17,0.997183,1490
5,317,MRO,22.54,0.967715,8873
6,195,FTNT,336.48,0.946771,594
7,345,NVDA,250.07,0.836273,799
8,109,COP,93.98,0.777753,2128
9,89,CF,81.90,0.767192,2442


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [137]:
hqm_columns = [
    'Ticker', 
    'Price', 
    'Number of Shares to buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats,quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol, 
                    data[symbol]['price'], 
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                ], index = hqm_columns
            ),ignore_index = True
        )
hqm_dataframe 

,Ticker,Price,Number of Shares to buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,140.35,N/A,0.105037,N/A,-0.229996,N/A,-0.129784,N/A,-0.00638136,N/A,N/A
1,AAL,17.79,N/A,-0.168133,N/A,-0.136525,N/A,-0.105415,N/A,0.0623572,N/A,N/A
2,AAP,214.29,N/A,0.308479,N/A,-0.0235794,N/A,-0.1181,N/A,-0.110306,N/A,N/A
3,AAPL,170.66,N/A,0.380654,N/A,0.117399,N/A,0.0196813,N/A,0.0338884,N/A,N/A
4,ABBV,152.64,N/A,0.459178,N/A,0.270814,N/A,0.277519,N/A,0.126739,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,128.46,N/A,0.206525,N/A,-0.0688645,N/A,-0.0204982,N/A,0.0189406,N/A,N/A
499,ZBH,128.80,N/A,-0.229543,N/A,-0.162503,N/A,-0.0278747,N/A,0.0490467,N/A,N/A
500,ZBRA,432.72,N/A,-0.163631,N/A,-0.298271,N/A,-0.306209,N/A,-0.118856,N/A,N/A
501,ZION,73.80,N/A,0.339384,N/A,0.265403,N/A,0.0597432,N/A,0.0750269,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [138]:
hqm_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 12 columns):
Ticker                           503 non-null object
Price                            503 non-null float64
Number of Shares to buy          503 non-null object
One-Year Price Return            499 non-null object
One-Year Return Percentile       503 non-null object
Six-Month Price Return           499 non-null object
Six-Month Return Percentile      503 non-null object
Three-Month Price Return         499 non-null object
Three-Month Return Percentile    503 non-null object
One-Month Price Return           499 non-null object
One-Month Return Percentile      503 non-null object
HQM Score                        503 non-null object
dtypes: float64(1), object(11)
memory usage: 47.3+ KB


In [139]:
hqm_dataframe.dropna(axis=0, how='any', inplace=True)
hqm_dataframe

,Ticker,Price,Number of Shares to buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,140.35,N/A,0.105037,N/A,-0.229996,N/A,-0.129784,N/A,-0.00638136,N/A,N/A
1,AAL,17.79,N/A,-0.168133,N/A,-0.136525,N/A,-0.105415,N/A,0.0623572,N/A,N/A
2,AAP,214.29,N/A,0.308479,N/A,-0.0235794,N/A,-0.1181,N/A,-0.110306,N/A,N/A
3,AAPL,170.66,N/A,0.380654,N/A,0.117399,N/A,0.0196813,N/A,0.0338884,N/A,N/A
4,ABBV,152.64,N/A,0.459178,N/A,0.270814,N/A,0.277519,N/A,0.126739,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,128.46,N/A,0.206525,N/A,-0.0688645,N/A,-0.0204982,N/A,0.0189406,N/A,N/A
499,ZBH,128.80,N/A,-0.229543,N/A,-0.162503,N/A,-0.0278747,N/A,0.0490467,N/A,N/A
500,ZBRA,432.72,N/A,-0.163631,N/A,-0.298271,N/A,-0.306209,N/A,-0.118856,N/A,N/A
501,ZION,73.80,N/A,0.339384,N/A,0.265403,N/A,0.0597432,N/A,0.0750269,N/A,N/A


In [140]:
hqm_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 499 entries, 0 to 502
Data columns (total 12 columns):
Ticker                           499 non-null object
Price                            499 non-null float64
Number of Shares to buy          499 non-null object
One-Year Price Return            499 non-null object
One-Year Return Percentile       499 non-null object
Six-Month Price Return           499 non-null object
Six-Month Return Percentile      499 non-null object
Three-Month Price Return         499 non-null object
Three-Month Return Percentile    499 non-null object
One-Month Price Return           499 non-null object
One-Month Return Percentile      499 non-null object
HQM Score                        499 non-null object
dtypes: float64(1), object(11)
memory usage: 50.7+ KB


In [141]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]
for row in hqm_dataframe.index:
    for time_period in time_periods:
        
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100
        #hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])
hqm_dataframe

,Ticker,Price,Number of Shares to buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,140.35,N/A,0.105037,0.410822,-0.229996,0.0621242,-0.129784,0.206413,-0.00638136,0.38477,N/A
1,AAL,17.79,N/A,-0.168133,0.0801603,-0.136525,0.184369,-0.105415,0.296593,0.0623572,0.769539,N/A
2,AAP,214.29,N/A,0.308479,0.763527,-0.0235794,0.462926,-0.1181,0.248497,-0.110306,0.0400802,N/A
3,AAPL,170.66,N/A,0.380654,0.841683,0.117399,0.783567,0.0196813,0.623246,0.0338884,0.625251,N/A
4,ABBV,152.64,N/A,0.459178,0.901804,0.270814,0.927856,0.277519,0.985972,0.126739,0.955912,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,128.46,N/A,0.206525,0.587174,-0.0688645,0.354709,-0.0204982,0.511022,0.0189406,0.543086,N/A
499,ZBH,128.80,N/A,-0.229543,0.0420842,-0.162503,0.132265,-0.0278747,0.486974,0.0490467,0.681363,N/A
500,ZBRA,432.72,N/A,-0.163631,0.0821643,-0.298271,0.0260521,-0.306209,0.0140281,-0.118856,0.0280561,N/A
501,ZION,73.80,N/A,0.339384,0.793587,0.265403,0.923848,0.0597432,0.705411,0.0750269,0.823647,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [142]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,Number of Shares to buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,140.35,N/A,0.105037,0.410822,-0.229996,0.0621242,-0.129784,0.206413,-0.00638136,0.38477,0.266032
1,AAL,17.79,N/A,-0.168133,0.0801603,-0.136525,0.184369,-0.105415,0.296593,0.0623572,0.769539,0.332665
2,AAP,214.29,N/A,0.308479,0.763527,-0.0235794,0.462926,-0.1181,0.248497,-0.110306,0.0400802,0.378758
3,AAPL,170.66,N/A,0.380654,0.841683,0.117399,0.783567,0.0196813,0.623246,0.0338884,0.625251,0.718437
4,ABBV,152.64,N/A,0.459178,0.901804,0.270814,0.927856,0.277519,0.985972,0.126739,0.955912,0.942886
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,128.46,N/A,0.206525,0.587174,-0.0688645,0.354709,-0.0204982,0.511022,0.0189406,0.543086,0.498998
499,ZBH,128.80,N/A,-0.229543,0.0420842,-0.162503,0.132265,-0.0278747,0.486974,0.0490467,0.681363,0.335671
500,ZBRA,432.72,N/A,-0.163631,0.0821643,-0.298271,0.0260521,-0.306209,0.0140281,-0.118856,0.0280561,0.0375752
501,ZION,73.80,N/A,0.339384,0.793587,0.265403,0.923848,0.0597432,0.705411,0.0750269,0.823647,0.811623


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [143]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True, drop=True)
hqm_dataframe

,Ticker,Price,Number of Shares to buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MOS,50.34,N/A,0.650384,0.973948,0.54914,0.975952,0.382056,0.997996,0.228872,0.993988,0.985471
1,MRO,22.02,N/A,0.965549,0.98998,0.896904,0.995992,0.307831,0.98998,0.128454,0.95992,0.983968
2,CF,79.80,N/A,0.762282,0.977956,0.760637,0.987976,0.227897,0.965932,0.119559,0.941884,0.968437
3,MCK,287.97,N/A,0.59914,0.961924,0.388661,0.963928,0.221979,0.957916,0.12122,0.943888,0.956914
4,DVN,56.20,N/A,1.79216,0.997996,0.952962,1,0.25037,0.97996,0.080572,0.839679,0.954409
5,LB,80.47,N/A,2.3444,1,0.838727,0.991984,0.222692,0.95992,0.0796682,0.833667,0.946393
6,ABBV,152.64,N/A,0.459178,0.901804,0.270814,0.927856,0.277519,0.985972,0.126739,0.955912,0.942886
7,ADM,81.10,N/A,0.425304,0.87976,0.332325,0.947896,0.197425,0.93988,0.160942,0.975952,0.935872
8,LYV,131.17,N/A,0.450096,0.895792,0.479721,0.973948,0.122051,0.847695,0.189975,0.985972,0.925852
9,HAL,32.96,N/A,0.504195,0.933868,0.634247,0.981964,0.404178,1,0.0651284,0.777555,0.923347


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [144]:
portfolio_input()

Enter the size of your portfolio10000000


In [145]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
hqm_dataframe

,Ticker,Price,Number of Shares to buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MOS,50.34,3972,0.650384,0.973948,0.54914,0.975952,0.382056,0.997996,0.228872,0.993988,0.985471
1,MRO,22.02,9082,0.965549,0.98998,0.896904,0.995992,0.307831,0.98998,0.128454,0.95992,0.983968
2,CF,79.80,2506,0.762282,0.977956,0.760637,0.987976,0.227897,0.965932,0.119559,0.941884,0.968437
3,MCK,287.97,694,0.59914,0.961924,0.388661,0.963928,0.221979,0.957916,0.12122,0.943888,0.956914
4,DVN,56.20,3558,1.79216,0.997996,0.952962,1,0.25037,0.97996,0.080572,0.839679,0.954409
5,LB,80.47,2485,2.3444,1,0.838727,0.991984,0.222692,0.95992,0.0796682,0.833667,0.946393
6,ABBV,152.64,1310,0.459178,0.901804,0.270814,0.927856,0.277519,0.985972,0.126739,0.955912,0.942886
7,ADM,81.10,2466,0.425304,0.87976,0.332325,0.947896,0.197425,0.93988,0.160942,0.975952,0.935872
8,LYV,131.17,1524,0.450096,0.895792,0.479721,0.973948,0.122051,0.847695,0.189975,0.985972,0.925852
9,HAL,32.96,6067,0.504195,0.933868,0.634247,0.981964,0.404178,1,0.0651284,0.777555,0.923347


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [174]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name="Momentum Strategy", index= False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [175]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [176]:
column_formats={
    'A': ['Ticker', string_template], 
    'B': ['Price', dollar_template], 
    'C': ['Number of Shares to buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [177]:
writer.save()